# 7. Function Decorators and Closures

- How python decides whether a variable is local
- Why closures exist and how they work
- What problem is solved by nonlocal
- How python evaluates decorator syntax
- Implementing a well-behaved decorator
- Interesting decorators in the standard library
- Implementing a parametrized decorator

## 7.3 Variable Scope rules

    - Python이 function을 compile할 때, function내의 변수를 local 변수로 인식한다.
    - 실제로 function을 call하면, function 내부에 정의되어 있지 않은 변수는 unbound 되었는지 확인하고, 존재하면 그 변수를 가져옴
    - Compile단계, 즉, function을 call하지 않을 경우, function 내부의 변수를 unbound 영역에서 찾을 수 없어도, 에러가 나지 않음

In [ ]:
from dis import dis

In [ ]:
def f1(a): # compile시에는 b를 local variable로 인식. 
    print(a)
    print(b)
# f1(3) # 실제로 call할 때, function내부에 정의되지 않은 변수를 unbound되었는지 확인하고, 존재하지 않아서 error

In [ ]:
dis(f1)

In [ ]:
b=6
def f2(a):
    print(a, b) # 함수 내의 local variable로 b가 함수 내부에는 존재하지 않으나, unbound 영역에서 존재 하므로 정상적으로 결과를 return
f2(3)

def f3(a):
    print(a, c)
c=6
f3(3) # 이와 같이 먼저 변수가 정의 되어 있지 않아도, 정상적으로 결과를 return

In [ ]:
dis(f2)

In [ ]:
dis(f3)

In [ ]:
def f4(a):
    print(a)
    print(b)
    b = 5
# f4(3)
# 컴파일시 a, b를 local variable로 인식한다. 실제로 여기서 함수를 call할 때, local var b를 함수 내부에서 찾았지만, 늦게 정의 됨
# local variable 'b' referenced before assignment 에러를 return

In [ ]:
b = 7
def f5(a): 
    global b # b를 global variable화 시킨다. compile할 때, b는 global variable로 인식
    print(a, b) # b가 local에서 뒤늦게 정의 되어 있지만, 앞에서 global이라고 선언했기 때문에, 전역에서 불러온다. 
    b = 9
b = 8
f5(3)

In [ ]:
dis(f5)

## 7.4 Closures
    - 정의: Non-global variable이 내부에서 선언되지는 않았지만, 이러한 variable을 참조하는 function
    - Function이 anonymous의 여부는 중요하지 않음. 중요한 건 내부에서 선언하지 않은 non-global variable을 참조하는가의 여부

In [ ]:
class Averager_class():
    def __init__(self):
        self.series=[]
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [ ]:
def Averager_func():
    series = [] 
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

In [ ]:
# Averager_func에 대해서, series는 local var이다. 그러나, 함수가 call될 경우, Averager_func가 이미 return을 함. local var이 더이상 아니게 됨
# averager에 대해서, series는 free variable이다.(해당 함수 안에 local var로 선언되지 않음)

In [ ]:
avg_class = Averager_class()
print(avg_class(10))
print(avg_class(11))
print(avg_class(12))

In [ ]:
avg_func = Averager_func()

In [ ]:
print(avg_func(10))
print(avg_func(11))
print(avg_func(12))
print(avg_func.__code__.co_varnames)
print(avg_func.__code__.co_freevars)

In [ ]:
print(avg_func.__code__.co_varnames)
print(avg_func.__closure__)
print(avg_func.__closure__[0]) # Binding of series is kept in the __closure__attribute of the returned function avg_func. (cell형태로)
print(avg_func.__code__.co_freevars) # Each item in avg_func.__closure__ corresponds to avg_code.__code__.co_freevars
print(avg_func.__closure__[0].cell_contents)

## 7.5 The nonlocal declaration
    - 위의 예제는 not efficient. --> Store all the values in the historical series and computed their avg sum everytiime avg_func is called
    - Better way: total과 # of item을 저장하고, 이들로부터 mean을 산출
    - 아래 예제를 이와 같이 실행하면, local var 'cnt' referenced before assignment라는 에러 발생
    - Why?
        - cnt는 number로 immutable type이다. 이는 실제로 averager의 body내에서 cnt에 값을 assign. --> cnt가 local variable화 되버림
        - 앞의 예제에서는 series가 free var였다. series.append 함수를 call 해서, sum과 len을 구했을 뿐, 실제로 값을 지정 안했음
        - 앞의 예제에서는 free variable이 되려고 한게 mutable했기 때문에 이런 상황이 가능했던 것
        - immutable type에서는 할수 있는 것은 read뿐, update는 할 수 없음.
        - Implicit하게 local var cnt를 생성하였으므로, 더이상 free var가 아니게 되어, closure에 저장되지 않게 되는 것

In [ ]:
def make_averager():
    cnt, total = 0, 0.0
    def averager(new_value):
        cnt += 1.0
        total += new_value
        return total / count
    return averager

In [ ]:
avg = make_averager()
avg(10)

    - 해결책: nonlocal을 사용하여, 이는 function 내부에서 새로운 값이 assign될때도 해당 variable이 free variable이라는 flag를 해준다

In [ ]:
def make_averager():
    cnt, total = 0, 0.0
    def averager(new_value):
        nonlocal cnt, total
        cnt += 1
        total += new_value
        return total / cnt
    return averager

In [ ]:
avg = make_averager()
avg(10)

## 7.1 Decorators 101

- Decorator: 다른 함수를 argument로 받는 callable (function)
- Decorator는 decorated function과 함께 어떠한 processing을 수행하여 이를 return하거나, 혹은 다른 함수나 callable object로 대체

In [ ]:
def deco(func):
    def inner():
        print("running inner()")
    return inner # deco는 inner function object를 return함

@deco
def target(): # target은 deco에 의해 decorated됨
    print("running target()")
    
target() # decorated target()을 invoke하면
target

- decorator의 특징중 하나는, decorated function이 defined된 직후, 바로 실행된다는 것이다.
- 만약 아래와 같은 코드를 registration.py라고 하고, 이를 다른 코드에서 import registration을 하면, decorator는 바로 실행된다!!!
- 그러나, decorated function은 실제로 불러야 실행이 된다.

In [ ]:
registry = []
def register(f):
    print('running register({})'.format(f))
    registry.append(f)
    return f

@register
def f1(): # 실제로 f1()과 f2()는 정의된 상태일 뿐, run되지 않았지만 decorator인 register의 print문을 실행함.
    print('running f1()')

@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')


In [ ]:
def main():
    print('running main()')
    print('registry -> {}'.format(registry))
    f1()
    f2()
    f3()

main()

- <b>How decorators are commonly used in real code??</b>
    - Decorator 함수는 decorated 함수와 같은 module안에 존재할 수 있다.
    - 일반적으로, decorator와 decorated 함수는 별도의 module에 존재한다.
    - 위의 예제에서, register decorator는 입력되는 함수와, return하는 함수가 동일하다. <b>그러나 일반적으로, decorator는 내부 함수를 정의하고 이를 return하는 형식으로 되어 있다.</b>

## 7.2 Decorator-enhanced Strategy Pattern

In [ ]:
from collections import namedtuple
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.quantity * self.price
    
class Order: # Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        self.__total = sum(lineitem.total() for lineitem in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            disc = 0
        else:
            disc = self.promotion(self)
        return self.total() - disc
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [ ]:
promos = []
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% disc for customers with fidelity point >= 1000 """
    return order.total()*0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% disc for each LineItem with 20 or more units"""
    disc = 0
    for item in order.cart:
        if item.quantity >= 20:
            disc += item.total() * 0.1
    return disc

@promotion
def large_order(order):
    """7% disc for orders with distinct item >= 10"""
    distinct_item = {item.product for item in order.cart}
    if len(distinct_item) >= 10:
        return order.total()*0.07
    return 0

def best_promo(order):
    """Select best disc available"""
    return max(promo(order) for promo in promos)

In [ ]:
Customer = namedtuple('Customer', 'name fidelity')
john = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('apple', 5, 2.0), LineItem('berry', 30, 0.5), LineItem('Chocolate', 30, 0.2)]
longcart = [LineItem(str(item_code), 1, 1.0) for item_code in range(20)]
print(Order(ann, cart, best_promo))
print(Order(john, cart, best_promo))
print(Order(ann, longcart, best_promo))
print(Order(john, longcart, best_promo))

## 7.6 Implementing a simple decorator

- <b>factorial(n)이 call될때마다, clocked(n)이 실행된다. 다음과 같은 순서로 실행이 된다.</b>
    - (1) start에 time.perf_counter()를 records
    - (2) factorial 함수를 call해서, save the result
    - (3) elapsed를 계산
    - (4) print format
    - (5) returns the result saved in (2)
- decorator는 decorated function을 same argument를 받는 새로운 function으로 대체함.

In [9]:
import time

def clock(func):
    """Measure the time elapsed"""
    def clocked(*args):
        start = time.perf_counter() # 1
        result = func(*args) # 2
        elapsed = time.perf_counter() - start # 3
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print("[{:.5f}s] {}({}) -> {}".format(elapsed, name, arg_str, result)) # 4
        return result # 5
    return clocked

In [10]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

print('*' * 20, 'Calling_snooze(0.5)')
snooze(.5)
print('*' * 20, 'factorial(15)')
factorial(15)
print('*' * 40)
print(factorial.__name__) # factorial 함수는 clocked function의 reference를 실제로 hold함

******************** Calling_snooze(0.5)
[0.50056s] snooze(0.5) -> None
******************** factorial(15)
[0.00000s] factorial(1) -> 1
[0.00023s] factorial(2) -> 2
[0.00043s] factorial(3) -> 6
[0.00054s] factorial(4) -> 24
[0.00066s] factorial(5) -> 120
[0.00077s] factorial(6) -> 720
[0.00097s] factorial(7) -> 5040
[0.00107s] factorial(8) -> 40320
[0.00124s] factorial(9) -> 362880
[0.00137s] factorial(10) -> 3628800
[0.00148s] factorial(11) -> 39916800
[0.00165s] factorial(12) -> 479001600
[0.00176s] factorial(13) -> 6227020800
[0.00187s] factorial(14) -> 87178291200
[0.00198s] factorial(15) -> 1307674368000
****************************************
clocked


- 위와 같은 clock decorator는 다음과 같은 한계가 존재
    - Keyword argument를 제공하지 않음
    - decorator function의 __name__, __doc__를 보여주지 않음

In [15]:
import time, functools
def clock2(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - start
        name = func.__name__
        arg_list = []
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['{} = {}'.format(k, w) for k, w in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(arg_list)
        print('[{:.6f}s] {}({}) -> {}'.format(elapsed, name, arg_str, result))
        return result
    return clocked

In [16]:
@clock2
def snooze(seconds):
    time.sleep(seconds)

@clock2
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

print('*' * 20, 'Calling_snooze(0.5)')
snooze(.5)
print('*' * 20, 'factorial(15)')
factorial(15)
print('*' * 40)
print(factorial.__name__) # factorial 함수는 clocked function의 reference를 실제로 hold함

******************** Calling_snooze(0.5)
[0.500563s] snooze(0.5) -> None
******************** factorial(15)
[0.000001s] factorial(1) -> 1
[0.000183s] factorial(2) -> 2
[0.000335s] factorial(3) -> 6
[0.000521s] factorial(4) -> 24
[0.000660s] factorial(5) -> 120
[0.000839s] factorial(6) -> 720
[0.000976s] factorial(7) -> 5040
[0.001111s] factorial(8) -> 40320
[0.001245s] factorial(9) -> 362880
[0.001379s] factorial(10) -> 3628800
[0.001514s] factorial(11) -> 39916800
[0.001647s] factorial(12) -> 479001600
[0.001780s] factorial(13) -> 6227020800
[0.001914s] factorial(14) -> 87178291200
[0.002102s] factorial(15) -> 1307674368000
****************************************
factorial


## 7.7 Decorators in the standard library

### 7.7.1 Memoization with functools.lru_cache

- lru_cache
    - 이전 단계에서 계산한 결과를 저장하여 해당 함수의 반복호출 시, computation time의 증가를 막아준다.

In [25]:
@clock2
def fibo1(n):
    if n<2:
        return n
    return fibo1(n-2) + fibo1(n-1)

@functools.lru_cache(maxsize=128, typed=False)
@clock2
def fibo2(n):
    if n<2:
        return n
    return fibo2(n-2) + fibo2(n-1)

In [26]:
print(fibo1(6))
print(fibo2(6))

[0.000001s] fibo1(0) -> 0
[0.000001s] fibo1(1) -> 1
[0.000468s] fibo1(2) -> 1
[0.000001s] fibo1(1) -> 1
[0.000001s] fibo1(0) -> 0
[0.000001s] fibo1(1) -> 1
[0.000294s] fibo1(2) -> 1
[0.000565s] fibo1(3) -> 2
[0.001143s] fibo1(4) -> 3
[0.000000s] fibo1(1) -> 1
[0.000000s] fibo1(0) -> 0
[0.000000s] fibo1(1) -> 1
[0.000074s] fibo1(2) -> 1
[0.000225s] fibo1(3) -> 2
[0.000000s] fibo1(0) -> 0
[0.000000s] fibo1(1) -> 1
[0.000218s] fibo1(2) -> 1
[0.000001s] fibo1(1) -> 1
[0.000000s] fibo1(0) -> 0
[0.000000s] fibo1(1) -> 1
[0.000215s] fibo1(2) -> 1
[0.000476s] fibo1(3) -> 2
[0.000987s] fibo1(4) -> 3
[0.001352s] fibo1(5) -> 5
[0.003026s] fibo1(6) -> 8
8
[0.000000s] fibo2(0) -> 0
[0.000001s] fibo2(1) -> 1
[0.000353s] fibo2(2) -> 1
[0.000001s] fibo2(3) -> 2
[0.000566s] fibo2(4) -> 3
[0.000001s] fibo2(5) -> 5
[0.000857s] fibo2(6) -> 8
8


### 7.7.2 Generic functions with single dispatch

- functools.singledispatch
    - From python 3.4
    - Allows each module to contribute to the overall solution
    - Easily provide a specialized function even for classes the you can't eidt
    - 일반 function에 이러한 @functools.singleddispatch를 하면, 첫번째 argument에 따라 같은 operation을 다른 방식으로 perform하게 할 수 있는 generic functin이 된다.

In [29]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(num):
    return '<pre> {0}(0x{0:x})</pre>'.format(num)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    content = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>{}</li>\n</ul>'.format(content)

In [34]:
print(htmlize({1,2,3}))
print(htmlize(abs))
print(htmlize('aaaaabbb'))
print(htmlize(16))
print(htmlize(['alpha, 66, {3,2,1}']))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<p>aaaaabbb</p>
<pre> 16(0x10)</pre>
<ul>
<li><p>alpha, 66, {3,2,1}</p></li>
</ul>


## 7.8 Stacked decorator

@d1
@d2

def f():
    print('f')
    
same as d1(d2(f))

## 7.9 Parametrized Decorators
### 7.9.1 A parametrized registration decorator

- Decorator는 1번째 argument로 decorated function을 받는다. 이 때, 다른 argument도 parameter로 받는 decorator는 어떻게 만드나?
    - 여러 개의 argument들을 다 받는 decorator factory를 만들어서, 이를 통해 decorator를 만든다

In [51]:
registry = set()
def register(active): # optional keyword argument를 받는다.
    def decorate(func): #실제 decorator의 역할. 자세히 보면, 실제 decorator처럼 func을 argument로 받는다.
        print("running register(active={}) -> decorate({})".format(active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func # decorate가 실제 decorator의 역할을 하기 때문에, func을 return해준다.
    return decorate # register는 decorator factory로, decorate를 return해준다.

In [52]:
@register(active=False)
def f1():
    print('running f1()')

@register(active=True)
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False) -> decorate(<function f1 at 0x7f08703d6e18>)
running register(active=True) -> decorate(<function f2 at 0x7f08703dbd90>)


In [53]:
registry

{<function __main__.f2>}

In [54]:
register(active=True)(f3)

running register(active=True) -> decorate(<function f3 at 0x7f08703d6c80>)


<function __main__.f3>

In [55]:
registry

{<function __main__.f3>, <function __main__.f2>}

In [56]:
register(active=False)(f2)

running register(active=False) -> decorate(<function f2 at 0x7f08703dbd90>)


<function __main__.f2>

In [57]:
registry

{<function __main__.f3>}

### 7.9.2 The parametrized clock decorator

In [85]:
import time
FMT1 = '[{elapsed: .4f}s] {name}({args}) -> {result}'
FMT2 = '{name}: {elapsed: .4f}s'
FMT3 = '{name}({args}) dt = {elapsed: .3f}s'

- 위의 예제에서 FMT안에는 4개의 {}가 있다.: {elapsed: .4f}, {name}, {args}, {result}
- fmt.format(**locals())는 fmt(=FMT)에서 {}에 해당하는 elapsed, name, args, result에 clocked의 local변수들을 각각 할당할 수 있도록 한다. 

In [86]:
def clock_param(fmt=FMT1): # clock_param: decorator factory
    def decorate(func): # 실제 decorator 역할
        def clocked(*_args): # Wrap decorated function
            start = time.time()
            _result = func(*_args) # decorated function의 결과
            elapsed = time.time() - start
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args) # _args는 clocked의 실제 argument. args는 display를 위한 str
            result = repr(_result) # display를 위한 _result의 string representation
            print(fmt.format(**locals())) # **locals()를 사용하여 fmt이 clocked의 모든 local variable을 참조할 수 있도록 함
            return result # clocked는 decorated func을 대체. --> return result를 함
        return clocked #clocked를 return
    return decorate

In [87]:
@clock_param(fmt=FMT1)
def snooze1(seconds):
    time.sleep(seconds)

@clock_param(fmt=FMT2)
def snooze2(seconds):
    time.sleep(seconds)

@clock_param(fmt=FMT3)
def snooze3(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze1(0.1)
for i in range(3):
    snooze2(0.1)
for i in range(3):
    snooze3(0.1)

[ 0.1002s] snooze1(0.1) -> None
[ 0.1001s] snooze1(0.1) -> None
[ 0.1005s] snooze1(0.1) -> None
snooze2:  0.1001s
snooze2:  0.1002s
snooze2:  0.1001s
snooze3(0.1) dt =  0.100s
snooze3(0.1) dt =  0.100s
snooze3(0.1) dt =  0.100s
